# LSTM


LSTM(Long Short-Term Memory)은 **순환신경망(RNN, Recurrent Neural Network)**의 일종으로, RNN의 단점인 **장기 의존성 문제**를 해결하기 위해 고안된 신경망 구조이다.

**LSTM의 주요 특징**
1. **장기 의존성(Long-Term Dependency) 처리**
   - 일반적인 RNN은 시간이 길어질수록 과거의 정보를 잘 기억하지 못하는 **기울기 소멸(Gradient Vanishing)** 문제가 발생한다.
   - LSTM은 **Cell State**와 **게이트 구조**를 통해 중요한 정보를 장기적으로 유지할 수 있다.

2. **게이트(Gates) 구조**
   - LSTM은 정보를 선택적으로 기억하거나 잊게 해주는 3가지 게이트로 구성된다:
     - **입력 게이트(Input Gate):** 새로운 정보를 얼마나 저장할지 결정한다.
     - **망각 게이트(Forget Gate):** 기존 정보를 얼마나 잊을지 결정한다.
     - **출력 게이트(Output Gate):** 현재 상태를 출력에 얼마나 반영할지 결정한다.

3. **Cell State**
   - 네트워크의 **기억 장치** 역할을 하며, 중요하지 않은 정보는 제거하고 중요한 정보는 유지한다.


**LSTM의 구조**

![](https://d.pr/i/iPf2jG+)

아래는 LSTM의 한 타임스텝(time step)에서 이루어지는 연산 과정이다:

1. **망각 게이트 (Forget Gate)**  
    * 이전 상태 $h_{t-1}$와 입력 $x_t$를 통해 제거할 정보를 결정한다.

$$
f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
$$

2. **입력 게이트 (Input Gate)**
    - 입력 게이트 $i_t$와 새로운 정보 $\tilde{C}_t$를 결합하여 Cell State에 반영할 정보를 생성한다.
    
$$
i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
$$

$$
\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)
$$

3. **Cell State 업데이트**  
    * 이전 Cell State $C_{t-1}$와 새로운 정보의 조합으로 현재 Cell State를 업데이트한다.

$$
C_t = f_t \cdot C_{t-1} + i_t \cdot \tilde{C}_t
$$

    

4. **출력 게이트 (Output Gate)**  
    * 출력 게이트 $o_t$와 업데이트된 Cell State $C_t$를 통해 새로운 은닉 상태 $h_t$를 계산한다.
    
$$
o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
$$

$$
h_t = o_t \cdot \tanh(C_t)
$$

**LSTM의 장점**

1. **장기 시퀀스 데이터 처리**: 시간의 흐름에 따라 발생하는 데이터를 잘 학습한다.
2. **텍스트, 음성, 시계열 데이터에 적합**: 언어 모델링, 번역, 주가 예측, 음성 인식 등 다양한 분야에서 활용된다.
3. **기울기 소멸 문제 해결**: Cell State와 게이트 구조 덕분에 학습이 안정적이다.

**추가 활용**

- **양방향 LSTM (Bidirectional LSTM)**: 양방향으로 데이터를 처리하여 더 많은 정보를 학습할 수 있다.
- **Stacked LSTM**: LSTM 레이어를 여러 층 쌓아 더 복잡한 패턴을 학습한다.

## LSTM 구조

In [3]:
# LSTM
import torch                         # PyTorch(텐서 연산/딥러닝) 라이브러리 불러오기
import torch.nn as nn                # 신경망 레이어(nn) 모듈을 nn이라는 별칭으로 불러오기

batch_size = 2                       # B : 한 번에 처리할 샘플(문장) 개수 = 2
seq_len = 3                          # T : 시퀀스 길이(타임스텝 수) = 3
input_size = 4                       # F : 각 타임스텝의 입력 특징(feature) 차원 = 4
hidden_size = 5                      # H : LSTM의 hidden state(은닉 상태) 차원 = 5

x = torch.randn(batch_size, seq_len, input_size)  # (B, T, F) 모양의 랜덤 입력 텐서 생성
print(x.shape)                       # 입력 텐서의 shape 출력

# RNN 생성 (B, T, F) 형태로 입력을 받도록 설정
lstm = nn.LSTM(input_size, hidden_size, batch_first=True)  # batch_first=True로 입력을 (B,T,F)로 받는 RNN 생성
output, (hidden, cell) = lstm(x)              # 입력 x를 RNN에 넣어 전체 출력(output)과 마지막 hidden(hidden) 얻기

print(output.shape)                  # output: 모든 타임스텝의 hidden state들 shape 출력 (B, T, H)
print(hidden.shape)                  # hidden: 마지막 타임스텝의 hidden state shape 출력 (num_layers, B, H)
print(cell.shape)                    # (Num_layers, B, H) : 마지막 시점 cell state


torch.Size([2, 3, 4])
torch.Size([2, 3, 5])
torch.Size([1, 2, 5])
torch.Size([1, 2, 5])


## IMDB 리뷰 감성분석
- IMDB 데이터는 영화 리뷰 텍스트와 그 리뷰의 감성(긍정/부정) 라벨로 구성된 이진 분류용 데이터셋
- 입력(X): 영화 리뷰 문장(원문 텍스트) → `tensorflow.keras.datasets.imdb`로 불러오면 “바로 모델에 넣어 실습할 수 있게” 미리 전처리된 형태(단어→정수 ID)로 제공
- 정답(y): 감성 라벨 0=부정, 1=긍정
- imdb.load_data(num_words=vocab_size)의 의미: 빈도 상위 vocab_size개 단어만 단어사전에 남기고, 나머지는(덜 나온 단어) 잘리거나 OOV로 처리되는 방식

- 결과적으로 **“영화 리뷰가 긍정인지 부정인지 맞추는 감성분류 데이터”**

In [4]:
from tensorflow.keras.datasets import imdb

vocab_size = 300                # 사용할 단어 사전 크기 (빈도 상위인 단어만 유지)

# IMDB 데이터 로드 (단어 ID 시퀀스 형태)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
print(X_train.shape, y_train.shape)     # 학습 데이터 개수
print(X_test.shape, y_test.shape)       # 테스트 데이터 개수

/opt/anaconda3/envs/nlp_env/lib/python3.11/site-packages/numpy/lib/_format_impl.py:838: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  array = pickle.load(fp, **pickle_kwargs)


(25000,) (25000,)
(25000,) (25000,)


imdb 단어사전 생성 및 리뷰 디코드

In [5]:
# 단어사전 생성
from keras.datasets import imdb                          # IMDB 데이터셋과 단어 인덱스를 제공하는 모듈

word_index = imdb.get_word_index()                       # 단어 → 정수 인덱스 매핑 딕셔너리 로드

pad_token = 0                                            # 패딩 토큰 인덱스
start_char = 1                                           # 문장 시작 토큰 인덱스
oov_char = 2                                             # 사전에 없는 단어(OOV) 토큰 인덱스
index_from = 2                                           # 실제 단어 인덱스가 시작되는 offset 값

# word_index(word->index)를 index->word 로 뒤집어서 생성
index_word = {                                           # 정수 인덱스 → 단어 매핑 딕셔너리 생성
    index + index_from: word                             # Keras IMDB는 index_from만큼 인덱스를 밀어서 사용
    for word, index in word_index.items()                # (단어, 인덱스) 쌍을 순회
    if index <= vocab_size                               # vocab_size 이하 단어만 사용
}

index_word[pad_token] = '[PAD]'                          # 0번 인덱스를 패딩 토큰으로 지정
index_word[start_char] = '[START]'                       # 1번 인덱스를 문장 시작 토큰으로 지정
index_word[oov_char] = '[OOV]'                           # 2번 인덱스를 미등록 단어 토큰으로 지정

index_word = dict(sorted(index_word.items(), key=lambda item: item[0]))  
# 인덱스 번호를 기준으로 정렬하여 가독성 있게 재구성

list(index_word.items())[:10]                            # 앞에서부터 10개의 (인덱스, 단어) 쌍 확인


[(0, '[PAD]'),
 (1, '[START]'),
 (2, '[OOV]'),
 (3, 'the'),
 (4, 'and'),
 (5, 'a'),
 (6, 'of'),
 (7, 'to'),
 (8, 'is'),
 (9, 'br')]

In [6]:
# 단어 ID 시퀀스를 텍스트 리뷰로 디코딩
decoded_review = ' '.join(index_word.get(i, "?") for i in X_train[0])
decoded_review

"[START] that on as about [OOV] [OOV] [OOV] [OOV] really [OOV] [OOV] see [OOV] and again who each a are any about [OOV] life what [OOV] [OOV] br they [OOV] everything a though and part life look [OOV] [OOV] [OOV] like and part [OOV] [OOV] for [OOV] from this [OOV] and take what as of those [OOV] movie that on and [OOV] [OOV] [OOV] and on me because i as about [OOV] from been was this [OOV] and on for [OOV] for i as [OOV] with [OOV] a which [OOV] i is [OOV] is two a and [OOV] [OOV] as [OOV] see [OOV] by and still i as from [OOV] a are off good who scene some are [OOV] by of on i come he bad more a that [OOV] as into [OOV] is and films best [OOV] was each and [OOV] to [OOV] a [OOV] who me about [OOV] [OOV] his [OOV] [OOV] has to and [OOV] [OOV] this characters how and [OOV] was american too at [OOV] no his something of enough [OOV] with and bit on film say [OOV] his [OOV] a back one [OOV] with good who he there's made are characters and bit really as from [OOV] how i as actor a as [OOV] 

정확도가 중요하면 vocab_size를 늘린다.

In [7]:
# 메모리 사용량 조절 train_size, test_size
train_size = 15000   # 학습 샘플 수
test_size = 10000    # 테스트 샘플 수 
X_train = X_train[:train_size]
y_train = y_train[:train_size]
X_test = X_test[:test_size]
y_test = y_test[:test_size]

print(X_train.shape, y_train.shape) # 학습 데이터 개수
print(X_test.shape, y_test.shape)   # 테스트 데이터 개수

(15000,) (15000,)
(10000,) (10000,)


In [8]:
# IMDB 시퀀스/라벨을 PyTorch Tensor로 변환
X_train = [torch.tensor(seq, dtype = torch.long) for seq in X_train]    # 학습 시퀀스(단어 ID 리스트) -> LongTensor 로 변환
X_test = [torch.tensor(seq, dtype = torch.long) for seq in X_test]      # 테스트 시퀀스(단어 ID 리스트) -> LongTensor 로 변환

y_train = torch.tensor(y_train, dtype = torch.float)    # 학습 라벨(0/1) -> FloatTensor로 변환
y_test = torch.tensor(y_test, dtype = torch.float)      # 테스트 라벨(0/1) -> FloatTensor로 변환

X_train[0] , X_train[0].shape

(tensor([  1,  14,  22,  16,  43,   2,   2,   2,   2,  65,   2,   2,  66,   2,
           4, 173,  36, 256,   5,  25, 100,  43,   2, 112,  50,   2,   2,   9,
          35,   2, 284,   5, 150,   4, 172, 112, 167,   2,   2,   2,  39,   4,
         172,   2,   2,  17,   2,  38,  13,   2,   4, 192,  50,  16,   6, 147,
           2,  19,  14,  22,   4,   2,   2,   2,   4,  22,  71,  87,  12,  16,
          43,   2,  38,  76,  15,  13,   2,   4,  22,  17,   2,  17,  12,  16,
           2,  18,   2,   5,  62,   2,  12,   8,   2,   8, 106,   5,   4,   2,
           2,  16,   2,  66,   2,  33,   4, 130,  12,  16,  38,   2,   5,  25,
         124,  51,  36, 135,  48,  25,   2,  33,   6,  22,  12, 215,  28,  77,
          52,   5,  14,   2,  16,  82,   2,   8,   4, 107, 117,   2,  15, 256,
           4,   2,   7,   2,   5,   2,  36,  71,  43,   2,   2,  26,   2,   2,
          46,   7,   4,   2,   2,  13, 104,  88,   4,   2,  15, 297,  98,  32,
           2,  56,  26, 141,   6, 194,   2,  18,   4

In [9]:
# 시퀀스 패딩 처리로 길이 고정
import torch.nn.functional as F    # padding 등 텐서 연산을 위한 PyTorch 함수 모듈

seq_len = 100                     # 모든 시퀀스를 맞출 목표 길이(최대 길이)

# 시퀀스들을 max_len 길이로 패딩(0) 또는 자르기하여 텐서로 변환
def pad_sequences(sequences, max_len):   # 시퀀스 리스트와 최대 길이를 입력으로 받는 함수
    padded_sequences = []                # 패딩된 시퀀스를 저장할 리스트
    for seq in sequences:                # 각 시퀀스에 대해 반복
        if len(seq) < max_len:            # 시퀀스 길이가 max_len보다 짧으면
            padded_seq = F.pad(seq, (0, max_len - len(seq)), value=0)  # 뒤쪽에 0으로 패딩
        else:                             # 시퀀스 길이가 max_len 이상이면
            padded_seq = seq[:max_len]    # max_len 길이만큼 자르기
        
        padded_sequences.append(padded_seq)  # 처리된 시퀀스를 리스트에 추가
    return torch.stack(padded_sequences)     # 리스트를 하나의 텐서로 변환하여 반환

X_train_padded = pad_sequences(X_train, seq_len)  # 학습 데이터 시퀀스를 길이 100으로 패딩
X_test_padded = pad_sequences(X_test, seq_len)    # 테스트 데이터 시퀀스를 길이 100으로 패딩

X_train_padded.shape, X_test_padded.shape          # 패딩된 학습/테스트 텐서의 shape 확인


(torch.Size([15000, 100]), torch.Size([10000, 100]))

In [10]:
# IMDB 감성분류 RNN 모델
class SentimentNet(nn.Module):                 # PyTorch nn.Module을 상속받은 RNN 모델 클래스 정의
    def __init__(self, input_dim, hidden_dim, output_dim):  # 입력·은닉·출력 차원을 받는 생성자
        super().__init__()                          # 부모 클래스(nn.Module) 초기화
        self.lstm = nn.LSTM(                          # RNN 레이어 정의
            input_size=input_dim,                   # 각 타임스텝 입력 벡터 차원
            hidden_size=hidden_dim,                  # 은닉 상태 차원
            batch_first=True                        # 입력 형태를 (B, T, F)로 사용
        )
        self.fc = nn.Linear(hidden_dim, output_dim) # 마지막 hidden을 출력 차원으로 변환하는 선형 레이어
    
    def forward(self, x):                           # 순전파 정의
        output, (hidden, cell) = self.lstm(x)       # output : (B,T,H), hidden / cell : (1:layers_num, B, H)
        output = self.fc(hidden[-1])                # 마지막 레이어 hidden -> (B, H) -> fc 적용
        return output                               # 최종 예측 결과 반환 (B, output_dim) 반환
    
input_dim = vocab_size                              # 단어 집합 크기(입력 차원)
hidden_dim = 16                                     # RNN 은닉 상태 차원
output_dim = 1                                      # 이진 분류를 출력(logit 1개)

model = SentimentNet(input_dim, hidden_dim, output_dim)  # 모델 생성
model                                             # 모델 구조 출력


SentimentNet(
  (lstm): LSTM(300, 16, batch_first=True)
  (fc): Linear(in_features=16, out_features=1, bias=True)
)